## 1. train

### 1.1 prepare data for training

In [ ]:
import sys
sys.path.append('/data3/fanpeishan/state/for_state/scripts')
from prepare_test_data import prepare_test_data
prepare_test_data(
    data_in="/data1/fanpeishan/State-Tahoe-Filtered/c37.h5ad",
    data_out="/data3/fanpeishan/state/for_state/data/State-Tahoe-Filtered-processed/c37_prep.h5ad",
    var_dims_path="/data3/fanpeishan/state/for_state/models/ST-Tahoe/var_dims.pkl"
)
prepare_test_data(
    data_in="/data1/fanpeishan/State-Tahoe-Filtered/c38.h5ad",
    data_out="/data3/fanpeishan/state/for_state/data/State-Tahoe-Filtered-processed/c38_prep.h5ad",
    var_dims_path="/data3/fanpeishan/state/for_state/models/ST-Tahoe/var_dims.pkl"
)

In [ ]:
加载测试数据: /data1/fanpeishan/State-Tahoe-Filtered/c37.h5ad
原始数据维度: (1835947, 62710)
模型高变基因数量: 2000
匹配基因数: 2000/2000
已删除所有obsm字段
转换 X 为稠密数组...
X_hvg 类型: <class 'numpy.ndarray'>
X_hvg 形状: (1835947, 2000)
保存测试数据: /data3/fanpeishan/state/for_state/data/State-Tahoe-Filtered-processed/c37_prep.h5ad
加载测试数据: /data1/fanpeishan/State-Tahoe-Filtered/c38.h5ad
原始数据维度: (2360694, 62710)
模型高变基因数量: 2000
匹配基因数: 2000/2000
已删除所有obsm字段
转换 X 为稠密数组...
X_hvg 类型: <class 'numpy.ndarray'>
X_hvg 形状: (2360694, 2000)
保存测试数据: /data3/fanpeishan/state/for_state/data/State-Tahoe-Filtered-processed/c38_prep.h5ad

### 1.2 train st

In [ ]:
export CUDA_VISIBLE_DEVICES=3
state tx train \
  name="run28_v4" \
  output_dir="/data3/fanpeishan/state/for_state/run_results/run28" \
  data.kwargs.toml_config_path="/data3/fanpeishan/state/for_state/run_commands/tomls/run28_v4.toml" \
  data.kwargs.embed_key=X_hvg \
  data.kwargs.num_workers=8 \
  data.kwargs.batch_col="plate" \
  data.kwargs.pert_col="drugname_drugconc" \
  data.kwargs.cell_type_key="cell_name" \
  data.kwargs.control_pert="\"[('DMSO_TF', 0.0, 'uM')]\"" \
  model=state_lg \
  training.batch_size=16 \
  training.max_steps=30000 \
  training.ckpt_every_n_steps=5000

In [ ]:
# run28_v4
# Dataset paths - maps dataset names to their directories
[datasets]
c37 = "/data3/fanpeishan/state/for_state/data/State-Tahoe-Filtered-processed/c37_prep.h5ad" 
c38 = "/data3/fanpeishan/state/for_state/data/State-Tahoe-Filtered-processed/c38_prep.h5ad" 

# Training specifications
# All cell types in a dataset automatically go into training (excluding zeroshot/fewshot overrides)
[training]
c37 = "train"
c38 = "train"

# Zeroshot specifications - entire cell types go to val or test
[zeroshot]

# Fewshot specifications - explicit perturbation lists
[fewshot]

[fewshot."c37.CVCL_1547"]
val = ["[('Almonertinib (mesylate)', 0.5, 'uM')]","[('Clonidine (hydrochloride)', 5.0, 'uM')]"]
test = ["[('Naproxen', 0.5, 'uM')]","[('Berberine (chloride hydrate)', 5.0, 'uM')]"]


# c37_prep：drugname_drugconc                        数量(前十个高频药物)
#[('DMSO_TF', 0.0, 'uM')]                         45150
#[('Adagrasib', 0.05, 'uM')]                      23449
#[('Afatinib', 0.5, 'uM')]                         6042
#[('Almonertinib (mesylate)', 0.5, 'uM')]          5277
#[('Clonidine (hydrochloride)', 5.0, 'uM')]        4556
#[('Naproxen', 0.5, 'uM')]                         4551
#[('Berberine (chloride hydrate)', 5.0, 'uM')]     4550
#[('Berbamine (dihydrochloride)', 0.5, 'uM')]      4480
#[('Belumosudil', 0.5, 'uM')]                      4465
#[('Gemfibrozil', 5.0, 'uM')]                      4410


## 2. test

### 2.1 prepare for infer

In [1]:
import sys
sys.path.append('/data3/fanpeishan/state/for_state/scripts')
from prepare_test_data import prepare_test_data
prepare_test_data(
    data_in="/data3/fanpeishan/state/for_state/data/State-Tahoe-Filtered-processed/c38_prep.h5ad",
    data_out="/data3/fanpeishan/state/for_state/run_results/run28/run28_v4/c38_step1.h5ad",
    var_dims_path="/data3/fanpeishan/state/for_state/run_results/run28/run28_v4/var_dims.pkl"
)

加载测试数据: /data3/fanpeishan/state/for_state/data/State-Tahoe-Filtered-processed/c38_prep.h5ad
原始数据维度: (2360694, 2000)
模型高变基因数量: 2000
匹配基因数: 2000/2000
已删除所有obsm字段
转换 X 为稠密数组...
X_hvg 类型: <class 'numpy.ndarray'>
X_hvg 形状: (2360694, 2000)
保存测试数据: /data3/fanpeishan/state/for_state/run_results/run28/run28_v4/c38_step1.h5ad


### 2.2 preprocess infer

In [ ]:
export CUDA_VISIBLE_DEVICES=2
state tx preprocess_infer \
  --adata /data3/fanpeishan/state/for_state/run_results/run28/run28_v4/c38_step1.h5ad \
  --output /data3/fanpeishan/state/for_state/run_results/run28/run28_v4/c38_step2.h5ad \
  --pert-col drugname_drugconc \
  --control-condition "[('DMSO_TF', 0.0, 'uM')]"

In [ ]:
Loading AnnData from /data3/fanpeishan/state/for_state/run_results/run28/run28_v4/c38_step1.h5ad
Set random seed to 42
Identifying control cells with condition: "[('DMSO_TF', 0.0, 'uM')]"
Found 51980 control cells out of 2360694 total cells
Processing 1136 non-control perturbations
Replacement complete! Replaced expression in 2308714 cells
Control cells (51980) retain their original expression
============================================================
PREPROCESSING SUMMARY:
  - Input: 2360694 cells, 2000 genes
  - Control condition: "[('DMSO_TF', 0.0, 'uM')]"
  - Control cells: 51980 (unchanged)
  - Perturbed cells: 2308714 (replaced with control expression)
  - Perturbations processed: 1136
  - Using expression matrix (X)

USAGE:
  The output file contains cells with control expression but original
  perturbation annotations. When passed through state_transition inference,
  the model will apply perturbation effects to simulate the original data.
  Compare: state_transition(output) ≈ original_input
============================================================
Saving preprocessed data to /data3/fanpeishan/state/for_state/run_results/run28/run28_v4/c38_step2.h5ad
Preprocessing complete!

### 2.3 infer

In [ ]:
export CUDA_VISIBLE_DEVICES=4
state tx infer \
    --model-dir /data3/fanpeishan/state/for_state/run_results/run28/run28_v4 \
    --checkpoint /data3/fanpeishan/state/for_state/run_results/run28/run28_v4/checkpoints/final.ckpt \
    --pert-col 'drugname_drugconc' \
    --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
    --batch-col 'plate' \
    --adata /data3/fanpeishan/state/for_state/run_results/run28/run28_v4/c38_step2.h5ad \
    --output /data3/fanpeishan/state/for_state/run_results/run28/run28_v4/c38_step3.h5ad \
    --quiet

In [ ]:
StateTransitionPerturbationModel(
  (loss_fn): SamplesLoss()
  (gene_decoder): LatentToGeneDecoder(
    (decoder): Sequential(
      (0): Linear(in_features=2000, out_features=1024, bias=True)
      (1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (2): GELU(approximate='none')
      (3): Dropout(p=0.1, inplace=False)
      (4): Linear(in_features=1024, out_features=1024, bias=True)
      (5): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (6): GELU(approximate='none')
      (7): Dropout(p=0.1, inplace=False)
      (8): Linear(in_features=1024, out_features=512, bias=True)
      (9): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (10): GELU(approximate='none')
      (11): Dropout(p=0.1, inplace=False)
      (12): Linear(in_features=512, out_features=2000, bias=True)
      (13): ReLU()
    )
  )
  (pert_encoder): Sequential(
    (0): Linear(in_features=1137, out_features=1488, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=1488, out_features=1488, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=1488, out_features=1488, bias=True)
    (7): GELU(approximate='none')
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=1488, out_features=1488, bias=True)
  )
  (basal_encoder): Linear(in_features=2000, out_features=1488, bias=True)
  (transformer_backbone): LlamaBidirectionalModel(
    (embed_tokens): Embedding(32000, 1488, padding_idx=0)
    (layers): ModuleList(
      (0-5): 6 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=1488, out_features=1488, bias=False)
          (k_proj): Linear(in_features=1488, out_features=1488, bias=False)
          (v_proj): Linear(in_features=1488, out_features=1488, bias=False)
          (o_proj): Linear(in_features=1488, out_features=1488, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=1488, out_features=5952, bias=False)
          (up_proj): Linear(in_features=1488, out_features=5952, bias=False)
          (down_proj): Linear(in_features=5952, out_features=1488, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((1488,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((1488,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((1488,), eps=1e-06)
    (rotary_emb): NoRoPE()
  )
  (project_out): Sequential(
    (0): Linear(in_features=1488, out_features=1488, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=1488, out_features=1488, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=1488, out_features=1488, bias=True)
    (7): GELU(approximate='none')
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=1488, out_features=2000, bias=True)
  )
  (final_down_then_up): Sequential(
    (0): Linear(in_features=2000, out_features=250, bias=True)
    (1): GELU(approximate='none')
    (2): Linear(in_features=250, out_features=2000, bias=True)
  )
  (relu): ReLU()
)

=== Inference complete ===
Input cells:         2360694
Controls simulated:  51980
Treated simulated:   2308714
Wrote predictions to adata.X
Saved:               /data3/fanpeishan/state/for_state/run_results/run28/run28_v4/c38_step3.h5ad
Saved count predictions to adata.X

### 2.4 eval

In [ ]:
export CUDA_VISIBLE_DEVICES=2
cell-eval run \
    -ap /data3/fanpeishan/state/for_state/run_results/run28/run28_v4/c38_step3.h5ad \
    -ar /data3/fanpeishan/state/for_state/run_results/run28/run28_v4/c38_step1.h5ad \
    -o /data3/fanpeishan/state/for_state/run_results/run28/run28_v4/run_results \
    --control-pert "[('DMSO_TF', 0.0, 'uM')]" \
    --pert-col 'drugname_drugconc' \
    --profile full \
    --batch-size 1024 \
    --num-threads 10

In [2]:
import pandas as pd
results=pd.read_csv('/data3/fanpeishan/state/for_state/run_results/run28/run28_v4/run_results/agg_results.csv')
mean_results=results[results.statistic == 'mean'][['overlap_at_100', 'pearson_delta','mse']]
print(mean_results)

   overlap_at_100  pearson_delta       mse
2        0.157251       0.809275  0.006153
